In [ ]:
# Text Summarization Inference and Testing
# This notebook loads a trained model and generates summaries for test documents

## Import Required Libraries

import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Configure TensorFlow logging to only show errors
tf.get_logger().setLevel(logging.ERROR)

# Disable tokenizer parallelism warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
## Configuration Parameters

# Inference Configuration
MAX_INPUT_LENGTH = 1024    # Maximum length of input text (tokens)
MIN_TARGET_LENGTH = 5      # Minimum length of generated summary (tokens)
MAX_TARGET_LENGTH = 128    # Maximum length of generated summary (tokens)
# Alternative: MAX_TARGET_LENGTH = 400  # For longer summaries

# Model Selection
# Specify the model checkpoint that matches your trained model
# MODEL_CHECKPOINT = "t5-small"              # For T5 models
MODEL_CHECKPOINT = "facebook/bart-base"      # For BART models

In [ ]:
## Load Trained Model

# Load the fine-tuned model from a local directory
# Make sure the path points to your saved model directory
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_path = "./t5_small"  # Update this path to your trained model location
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)
print(f"Model loaded from: {model_path}")
print(f"Model type: {type(model).__name__}")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./t5_small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
## Compile Model (Optional for Inference)

# Compiling the model is not strictly necessary for inference,
# but it ensures the model is ready if you want to continue training
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)
print("Model compiled successfully!")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
## Load Tokenizer

# Load the tokenizer corresponding to the model architecture
# The tokenizer must match the model (T5 or BART)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
print(f"Tokenizer loaded: {MODEL_CHECKPOINT}")

In [ ]:
## Configure Model-Specific Prefix

# T5 models require a task prefix for text-to-text tasks
# BART models don't need a prefix
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "  # T5 prefix for summarization task
else:
    prefix = ""  # No prefix for BART models

print(f"Using prefix: '{prefix}'")

In [21]:
#example = "Fresh shelling from Ukraine rocked Belgorod overnight, the governor said in a video posted Sunday morning, as Russian dissidents ramp up pressure on the western border region.Gov. Vyacheslav Gladkov said there had been Ukrainian attacks in several locations under his administration.`The night was rather turbulent,` Gladkov said. `There is a lot of destruction. There is no information about casualties.`Due to the violence, 4,000 people are being housed in temporary accommodations. Children in the area are being moved to a camp in Crimea for their own safety, Gladkov added.Dissidents appear near shelled area: Also Sunday, the Freedom for Russia Legion, one of two dissident Russian units fighting under Ukrainian command, posted a video which they said showed their fighters on the streets of a village on the outskirts of Shebekino, one of the areas Gladkov said was attacked.The footage appeared to show the legion in Novaya Tavolzhanka, according to geolocation by CNN, and groups of people moving through the streets as a unit.“We’re going in! The advance assault group of the Legion and the Russian Volunteer Corp entering the suburb of Shebekino,” the group said in the clip's caption.CNN cannot verify the legion’s claim, but the video’s release will be seen as a further attempt to destabilize Russia in the information space as well as disrupting its military plans.Meetings requested: In another bold move, the legion posted a video in which its leader and that of a second dissident group, the Russian Volunteer Corps, request a meeting with Gladkov. In exchange, they offered to release two Russian soldiers allegedly in their custody.The video shows the purported soldiers giving their names and those of their hometowns in Russia. The dissident leaders -- who have made no secret of their opposition to Russian President Vladimir Putin -- say they want to talk to Gladkov about the fate of the country and the war. No threat is made to the lives of the men they are holding."

In [ ]:
## Add Prefix to Input (if needed)

# Add the task prefix for T5 models (BART models don't need this)
example_pref = prefix + example
print(f"Input text with prefix: {len(example_pref)} characters")

In [ ]:
## Tokenize Input Text

# Convert the input text to token IDs that the model can process
model_input = tokenizer(
    example_pref, 
    max_length=MAX_INPUT_LENGTH, 
    truncation=True
)

print(f"Tokenized input length: {len(model_input['input_ids'])} tokens")
print(f"Input IDs shape: {model_input['input_ids']}")

In [ ]:
## Generate Summary

# Create a summarization pipeline for easy inference
from transformers import pipeline

summarizer = pipeline(
    "summarization", 
    model=model, 
    tokenizer=tokenizer, 
    framework="tf"
)

# Generate summary with specified length constraints
print("Generating summary...")
print("="*50)
result = summarizer(
    example,
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

# Display results
print("\nGenerated Summary:")
print("-" * 50)
print(result[0]['summary_text'])
print("\n" + "="*50)
print("\nOriginal Text (first 300 characters):")
print("-" * 50)
print(example[:300] + "...")

[{'summary_text': "MG Rover's proposed tie-up with Chinese carmaker Shanghai Automotive could result in 3,000 jobs being lost if the deal goes ahead, according to the Financial Times."}]